In [1]:
# Standard Library Imports (if any)

# Third-party Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from IPython.display import display, HTML
from adjustText import adjust_text
from sklearn.preprocessing import StandardScaler

# Local Module Imports
from utils import *
from data_handling import *
from plot import *
from data_handling.data_processor import *

In [2]:
major = COUNTRIES.MAJOR
offshore = COUNTRIES.OFFSHORE
sample = major + offshore
period = (2001, 2004)
dm = DataManager(
    raw_dir = "../data/raw",
    save_dir = "../data/clean"
)
ds_full = dm.get_dataset("ds")
fed_full = dm.get_dataset("fed")
# dm.clean_data()
# dm.save_data()
dm.filter_data(None, period)

In [3]:
# Access datasets
cpis = dm.get_dataset("cpis")
ds = dm.get_dataset("ds")
fed = dm.get_dataset("fed")
wb = dm.get_dataset("wb")
wfe = dm.get_dataset("wfe")
gdp = dm.get_dataset("gdp")

major = COUNTRIES.MAJOR
offshore = COUNTRIES.OFFSHORE
sample = major + offshore

In [9]:
weights_per_year = compute_weights(cpis, wb, major, offshore)
weights_dict = {}
for year in weights_per_year.columns:
    weights_dict[year] = weights_per_year[year].unstack("Country")
weights_avg = weights_per_year.mean(axis=1).unstack("Country")
index_returns = compute_index_excess_returns(ds, fed ,major)

means = pd.DataFrame(index=major)
vars = pd.DataFrame(index=major)
covs = pd.DataFrame(index=major)

for year, weights in weights_dict.items():
    returns = compute_excess_returns_statistics(index_returns, weights)
    means[year] = returns["mean_portfolio"]
    vars[year] = returns["var_portfolio"]  
    covs[year] = returns["cov_index_portfolio"]

returns = compute_excess_returns_statistics(index_returns, weights_avg)
means_avg = returns["mean_portfolio"]
vars_avg = returns["var_portfolio"]  
covs_avg = returns["cov_index_portfolio"]

In [11]:
print((means.var(axis=1) / (vars_avg / 12)).mean())

0.0006574869835458848


In [1]:
# Standard Library Imports (if any)

# Third-party Library Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from IPython.display import display, HTML
from adjustText import adjust_text
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Local Module Imports
from data_handling import *
from data_handling.data_processor import *
from plot import *
from utils import *

major = COUNTRIES.MAJOR
offshore = COUNTRIES.OFFSHORE
sample = major + offshore
period = (2001, 2004)
dm = DataManager(
    raw_dir = "../data/raw",
    save_dir = "../data/clean"
)
ds_full = dm.sources["ds"].filter_period((None, 2004))
fed_full = dm.sources["fed"].filter_period((None, 2004))
dm.filter_data(None, period)

# Access datasets
cpis = dm.get_dataset("cpis")
ds = dm.get_dataset("ds")
fed = dm.get_dataset("fed")
wb = dm.get_dataset("wb")
wfe = dm.get_dataset("wfe")
gdp = dm.get_dataset("gdp")
gdppc = dm.get_dataset("gdppc")

index_excess_returns = compute_index_excess_returns(ds_full, fed_full, major)
weights = compute_weights(cpis, wb, major, offshore).mean(axis=1).unstack(level="Country")
return_statistics = compute_excess_returns_statistics(index_excess_returns, weights)
domestic_investments = compute_domestic_investments(cpis, wb, major).droplevel(1)
y = domestic_investments / gdp; y = y.mean(axis=1)
X_1 = gdppc.mean(axis=1)
X_2 = return_statistics["mean_portfolio"]
X_3 = return_statistics["var_portfolio"]
X = pd.concat([X_1, X_2, X_3], axis=1, keys=["gdp/cap", "mean", "var"])

X=(X-X.mean())/X.std() # Normalize
X = sm.add_constant(X).sort_index()

model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": 4}) # Newey-West with 4 lags 
y_pred = model.predict(X)

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

print(model.summary())

   feature       VIF
0    const  1.000000
1  gdp/cap  1.147939
2     mean  2.737717
3      var  2.883875
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.810
Date:                Sat, 05 Apr 2025   Prob (F-statistic):            0.00690
Time:                        16:55:02   Log-Likelihood:                -29.967
No. Observations:                  37   AIC:                             67.93
Df Residuals:                      33   BIC:                             74.38
Df Model:                           3                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from arch import arch_model

# Assume returns_df contains country-level stock returns
# and world_return is the global stock market return.

# Construct the independent variables
X = pd.DataFrame()
X["world_return"] = world_return
X["gdp_per_cap"] = gdp / wb.loc[major]
X["var"] = ds.var()  # Could use rolling window variance
X["gdp_growth"] = gdp.pct_change()  # GDP growth as proxy for expected future growth
X = X.dropna()

# Fit a GARCH(1,1)-M model
returns = ds.loc[X.index]  # Align data
garch_m = arch_model(returns, vol="Garch", p=1, q=1, mean="LS")
garch_m_fit = garch_m.fit(disp="off")

# Extract expected return (fitted value)
expected_returns = garch_m_fit.params["mu"] + X @ garch_m_fit.params.iloc[1:]


NameError: name 'world_return' is not defined